# Gibbs energy minimization (fixed T, P, bulk composition)
Closed system; crystallization of a rhyolitic liquid using rhyolite-MELTS

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys

In [ ]:
pwd

In [ ]:
from thermoengine import core, phases, model, equilibrate

In [ ]:
#%run "core.ipynb"
%run "load_coder_fitted_models.ipynb"

In [ ]:
os.chdir('/Users/jennaadams/Documents/projects/thermoengine_mirrored/https:/gitlab.com/aswolf/ThermoEngineMirror/Notebooks/Calibration/Garnet_calibration')

## Create phases for equilibrium assemblages

In [ ]:
modelDB = model.Database(liq_mod='pMELTS')

In [ ]:
modelDB_melts = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
#Liquid_melts = modelDB_melts.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')
Rutile = modelDB.get_phase('Rt')
Cpx = modelDB.get_phase('Cpx')
garnet_pmelts=modelDB.get_phase('Grt')


In [ ]:
best_fit_garnet_vals=np.array([ 298.15, 1, 5.19104094e+04,  3.21504843e+01,  1.20993363e-01,  9.28034371e+03,
        3.67696598e+01, -4.54660626e-01,  1.00111240e+04, -8.71817781e+00,
        3.27875343e-02,  1.22156445e+03,  1.74981805e+00, -6.89189413e-01,
        5.98742483e+04,  1.82413452e+01,  4.88655990e-02, -2.32001967e+03,
        6.21265452e+00, -4.17161548e-01])

In [ ]:
len(best_fit_garnet_vals)

In [ ]:
grt=modelDB_fitted_pmelts.phases['Grt']

In [ ]:
grt.get_param_values()

In [ ]:
param_names=grt.param_names[:-3]

In [ ]:
len(param_names)

In [ ]:
#grt.set_param_values(best_fit_garnet_vals, param_names=param_names)

In [ ]:
def set_param_values(param_names, param_vals, params_to_change, modelDB=None):
    
    assert len(param_vals)==len(params_to_change), 'length of param_vals must equal length of params_to_change'
    
    grt_model=modelDB.phases['Grt']
    
    param_names_index=[]
    for iparam in params_to_change:
        param_names_index.extend([param_names.index(iparam)])
    #param_names_index=[param_names.index(params_to_change)]
    
    grt_model.set_param_values(param_vals, param_names_index)

In [ ]:
#param_names=fitted_coder.param_names
params_to_change=param_names

iparam_vals=best_fit_garnet_vals
set_param_values(param_names, iparam_vals, params_to_change, modelDB=modelDB_fitted_pmelts)
fitted_grt=modelDB_fitted_pmelts.phases['Grt']

In [ ]:
fitted_grt.get_param_values()

In [ ]:
Rutile.get_param_values()

In [ ]:
#Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

## Define elements in system and phases in system

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
#elm_sys = ['O','Na','Mg','Al','Si','K','Ca','Ti','Mn','Fe']
#phs_sys = [Liquid, Feldspar, Quartz, Rutile, Cpx, fitted_grt]
phs_sys = [Liquid, Feldspar, Quartz, Rutile, Cpx, garnet_pmelts]

## Composition of the system
This is an oceanic crust-like composition

In [ ]:
grm_oxides = {
    'SiO2':  50.05, 
    'TiO2':   1.97, 
    'Al2O3': 15.76, 
    'Fe2O3':  1.35,
    'Cr2O3':  0.0, 
    'FeO':    8.136, 
    'MnO':    0.17,
    'MgO':    7.9, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    11.74, 
    'Na2O':   3.04, 
    'K2O':    0.03, 
    'P2O5':   0.0, 
    'H2O':    0
}

Cast this composition as moles of elements for input to the Equilibrate class

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)

In [ ]:
Liquid.calc_endmember_comp?

In [ ]:
pwd

In [ ]:
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxides, method='intrinsic', output_residual=True)

In [ ]:
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
#moles_end = Liquid.calc_endmember_comp(
#    mol_oxides, method='intrinsic')
#print(np.array(moles_end[0]))
if not Liquid.test_endmember_comp(moles_end[0]):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end[0],output='moles_elements')

In [ ]:
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)

In [ ]:
blk_cmp

## Instantiate class instance and run calculation

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
t = 1250.0
p = 20000.0
#state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0, stats=True)
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

Pickup runs use previously computed state

In [ ]:
state = equil.execute(t-20.0, p, state=state, debug=0, stats=True)
state.print_state()

In [ ]:
state = equil.execute(t-25.0, p, state=state, debug=0, stats=True)
state.print_state()